# Introduction:

This ipynb file is used to build a model for a time series classification problem. 
<br>

We will start by : 
<br>
1- Importing the training datasets.
<br>
2- Merge the two datasets 
<br>
3- Add new features (Mean, median, etc...)

In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

dfX= pd.read_csv("InputTrain.csv")
dfY = pd.read_csv("StepOne_LabelTrain.csv")


In [132]:
import math
import re
#l'ajout manuel des features prend un peu de temps c'est pas opti, ~50sec pour la cellule entière

# Ajout manuel de 3 features (qui utilisent des Series et pas le dataframe entier)
autocor = []
idmax = []
entropy = []
for i in range(0,10421) :
  autocor.append(dfX.iloc[i,2:2161].autocorr(216))
  idmax.append((int)(re.search(r'\d+',dfX.iloc[i,2:2161].idxmax())[0]))
  entropy.append(sum([ p * math.log(p) / math.log(2.0) for p in dfX.iloc[i,2:2161]]))
dfX['autocor'] = autocor
dfX['idmax'] = idmax
dfX['entropy'] = entropy

# Ajout automatique de 6 features (qui utilisent le dataframe entier)
dfX['mean'] = dfX.iloc[:, 2:2161].mean(axis=1)

dfX['median'] = dfX.iloc[:, 2:2161].median(axis=1)

dfX['std'] = dfX.iloc[:, 2:2161].std(axis=1)

dfX['var'] = dfX.iloc[:, 2:2161].var(axis=1)

dfX['skew'] = dfX.iloc[:, 2:2161].skew(axis=1)

dfX['sem'] = dfX.iloc[:, 2:2161].sem(axis=1)

dfX.fillna(0,inplace=True)

In [133]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM

# Importer les données d'entrainement
train_input = dfX.drop(["Index"],axis=1)
train_labels = pd.read_csv('StepOne_LabelTrain.csv').iloc[:, 2:7]

# Normaliser les données d'entrée
scaler = MinMaxScaler()
train_input_scaled = scaler.fit_transform(train_input.iloc[:, 2:])
train_input = np.expand_dims(train_input_scaled, axis=2)

# Créer les 5 modèles de classification binaire pour chaque appareil
models = []
for i in range(5):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(train_input.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(32, return_sequences=True))
    model.add(LSTM(16, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_input, train_labels.iloc[:, i], epochs=5, batch_size=64, validation_split=0.2)
    models.append(model)

Epoch 1/5
131/131 [==============================] - 34s 67ms/step - loss: 0.4344 - accuracy: 0.7988 - val_loss: 0.4332 - val_accuracy: 0.8082
Epoch 2/5
131/131 [==============================] - 7s 54ms/step - loss: 0.3931 - accuracy: 0.8205 - val_loss: 0.4330 - val_accuracy: 0.8091
Epoch 3/5
131/131 [==============================] - 8s 60ms/step - loss: 0.3732 - accuracy: 0.8252 - val_loss: 0.4391 - val_accuracy: 0.7976
Epoch 4/5
131/131 [==============================] - 8s 62ms/step - loss: 0.3518 - accuracy: 0.8385 - val_loss: 0.4480 - val_accuracy: 0.7789
Epoch 5/5
131/131 [==============================] - 7s 55ms/step - loss: 0.3269 - accuracy: 0.8521 - val_loss: 0.4332 - val_accuracy: 0.7909
Epoch 1/5
131/131 [==============================] - 13s 61ms/step - loss: 0.3389 - accuracy: 0.8436 - val_loss: 0.2725 - val_accuracy: 0.8825
Epoch 2/5
131/131 [==============================] - 8s 61ms/step - loss: 0.2769 - accuracy: 0.8678 - val_loss: 0.2698 - val_accuracy: 0.8710
Epoc

In [137]:
test_data = pd.read_csv("InputTest.csv")

# Ajout manuel de 3 features (qui utilisent des Series et pas le dataframe entier)
autocor = []
idmax = []
entropy = []
for i in range(0,2488) :
  autocor.append(test_data.iloc[i,2:2161].autocorr(216))
  idmax.append((int)(re.search(r'\d+',test_data.iloc[i,2:2161].idxmax())[0]))
  entropy.append(sum([ p * math.log(p) / math.log(2.0) for p in test_data.iloc[i,2:2161]]))
test_data['autocor'] = autocor
test_data['idmax'] = idmax
test_data['entropy'] = entropy

# Ajout automatique de 6 features (qui utilisent le dataframe entier)
test_data['mean'] = test_data.iloc[:, 2:2161].mean(axis=1)
test_data['median'] = test_data.iloc[:, 2:2161].median(axis=1)
test_data['std'] = test_data.iloc[:, 2:2161].std(axis=1)
test_data['var'] = test_data.iloc[:, 2:2161].var(axis=1)
test_data['skew'] = test_data.iloc[:, 2:2161].skew(axis=1)
test_data['sem'] = test_data.iloc[:, 2:2161].sem(axis=1)

test_input = test_data.drop(["Index"],axis=1)

# Normaliser les données de test
scaler = MinMaxScaler()
test_input_scaled = scaler.fit_transform(test_input.iloc[:, 2:])

test_data.fillna(0,inplace=True)

In [138]:
# Faire des prédictions avec les 5 modèles pour les données de test
predictions = []
for model in models:
    prediction = model.predict(np.expand_dims(test_input_scaled, axis=2))
    predictions.append(prediction.flatten())

# Convertir les prédictions en un tableau de dimension (len(test_input), 5)
class_preds = np.zeros((len(test_input), 5))
for i in range(5):
    class_preds[:, i] = (predictions[i] > 0.1).astype(int)

# Enregistrer les prédictions dans un fichier CSV
output_df = pd.DataFrame(class_preds, columns=["Washing Machine", "Dishwasher", "Tumble Dryer", "Microwave", "Kettle"])
output_df.insert(0, "Index", test_data["Index"])
output_df.to_csv("./new_features4.csv", index=False)


78/78 [==============================] - 2s 21ms/step
